In [1]:
%env CUDA_VISIBLE_DEVICES=1
%env HF_HOME=/mnt/LLM
%env OMP_NUM_THREADS=16

import sys; sys.path.insert(0, "../.");

import torch
import transformers

from async_reasoning.async_reasoning_generation import Solver

MODEL_NAME = "Qwen/Qwen3-32B"  # for 48GB gpus, use "Qwen/Qwen3-32B-AWQ" instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = transformers.AutoTokenizer.from_pretrained(MODEL_NAME)
model = transformers.AutoModelForCausalLM.from_pretrained(
    MODEL_NAME, torch_dtype='auto', low_cpu_mem_usage=True, device_map=device)

# forbidden_token_ix are Model-dependent
forbidden_token_ix = [tokenizer.vocab[x] for x in ("</think>", "<|im_start|>", "SYSTEM")]


env: CUDA_VISIBLE_DEVICES=1
env: HF_HOME=/mnt/LLM
env: OMP_NUM_THREADS=16


/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/usr/bin/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status


Loading checkpoint shards:   0%|          | 0/17 [00:00<?, ?it/s]

In [2]:
problem = """Calculate x - x^2 + x^3 for x = 5,6,7,8. Return all 4 answers in \\boxed{ }."""
solver = Solver(model, tokenizer, forbidden_token_ix, use_fast_kernel=True)
writer_putput_str, thinker_output_str, token_times, eos_generated = solver.solve_with_async_reasoning(problem, budget=1024, display_generation_in_real_time=True)

# State.thinker_and_writer

## Thinker mode


<think>
I am in Thinker mode. My text is not visible to the user. I reason continuously, examining the visible writing above and refining the ideas behind it. I detect errors, test assumptions, and plan improvements. I express thoughts naturally, marking when something should change or be expanded. My goal is to keep reasoning clear, evolving, and supportive of strong written output.

Okay, let's see. The user wants me to calculate the expression x - x² + x³ for x values 5, 6, 7, and 8. I need to compute each one step by step.

Starting with x = 5. The expression is 5 - 5² + 5³. Let's break it down: 5 squared is 25, and 5 cubed is 125. So substituting those in, it becomes 5 - 25 + 125. Calculating that: 5 - 25 is -20, and then adding 125 gives 105. So for x=5, the result is 105.

Next, x = 6. The expression is 6 - 6² + 6³. 6 squared is 36, and 6 cubed is 216. Substituting those in: 6 - 36 + 216. 6 - 36 is -30, and adding 216 gives 186. So for x=6, the result is 186.

Now x = 7. The expression is 7 - 7² + 7³. 7 squared is 49, and 7 cubed is 343. Substituting: 7 - 49 + 343. 7 - 49 is -42, and adding 343 gives 301. So for x=7, the result is 301.

Finally, x = 8. The expression is 8 - 8² + 8³. 8 squared is 64, and 8 cubed is 512. Substituting: 8 - 64 + 512. 8 - 64 is -56, and adding 512 gives 456. So for x=8, the result is 456.

Let me double-check each calculation to make sure I didn

## Writer mode


I am in Writer mode. My text is visible to the user. I focus on clear, precise expression and careful word choice. I write only what is well-reasoned and verified in my workspace. I never speculate or improvise. If my thinking shifts or reveals an error, I immediately adjust. My goal is calm, accurate, and readable output.

We are asked to evaluate the expression $ x - x^2 + x^3 $ for $ x = 5, 6, 7, 8 $. Let's compute each value step by step.

---

**For $ x = 5 $:**

$$
5 - 5^2 + 5^3 = 5 - 25 + 125 = 105
$$

---

**For $ x = 6 $:**

$$
6 - 6^2 + 6^3 = 6 - 36 + 216 = 186
$$

---

**For $ x = 7 $:**

$$
7 - 7^2 + 7^3 = 7 - 49 + 343 = 201
$$

---

**For $ x = 8 $:**

$$
8 - 8^2 + 8^3 = 8 - 64 + 512 = 456
$$

---

The results are:

$$
\boxed{105, 186, 201, 456}
$$<|im_end|>